In [1]:
# Run if working locally
%load_ext autoreload
%autoreload 2
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import sqlite3
from sqlite3 import Error
import pickle
import os, sys
import config

config.root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, config.root_path)

from db.dbv2 import Table, AugmentedTable, TrainTestTable

<IPython.core.display.Javascript object>

In [3]:
from src.dataset.gpt_augmentor import Augmentor

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

Metal device set to: Apple M1


2023-01-09 21:19:21.763184: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-09 21:19:21.763427: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


<IPython.core.display.Javascript object>

In [4]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize

<IPython.core.display.Javascript object>

In [14]:
dataset_type = "committee"

table = Table(dataset_type)
augmented_table = AugmentedTable(dataset_type)
train_test_table = TrainTestTable(dataset_type)

<IPython.core.display.Javascript object>

In [15]:
min_sent_tokens = 8
max_sent_tokens = 64

target_sentences_original = table.get_target_sentences()
target_sentences = [s[1] for s in target_sentences_original]
cleaned_target_sentences = []

for s in target_sentences:
    if len(word_tokenize(s)) > min_sent_tokens:
        shortened_sentence = " ".join(word_tokenize(s)[:max_sent_tokens])
        cleaned_target_sentences.append(shortened_sentence)

augmented_segments = Augmentor.augment_gpt2(
    cleaned_target_sentences[:2],
    fast=True,
    # multiply by 5 to account for 5 as a max segment
    max_seq_word_length=max_sent_tokens * 5,
    verbose=True,
)

Completed augmenting 1/2...
Completed augmenting 2/2...


<IPython.core.display.Javascript object>

In [16]:
augmented_segments

[["thank you very much for that question, chair. i've got to say 'yes ', in many regards. so, the key focus of the programme in the early stages was about improving access to specialist child and adolescent mental health services. we developed the windscreen model—or we lifted the windscreen model. other models that are very similar have been used in our research. but, it's important to note that this model was first used by the National Institute of Mental Health and its predecessor, the Children's Mental Hospital (MHC) in England and Wales. We used this approach in other areas too, including, in a very short period of time, to develop the first comprehensive model of child mental care. The model used was based on a comprehensive assessment of mental illness and the impact it has on children. It has also been adapted for use by other health care organisations, notably the NHS Trust, which has been using it for over 10 years. The new model uses a more detailed",
  "thank you very much 

<IPython.core.display.Javascript object>

## Testing generated sentences

In [10]:
# https://www.kaggle.com/code/tuckerarrants/text-generation-with-huggingface-gpt2/notebook
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer, GPT2TokenizerFast

<IPython.core.display.Javascript object>

In [11]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

gpt_model = TFGPT2LMHeadModel.from_pretrained(
    "gpt2", pad_token_id=tokenizer.eos_token_id
)

sentence = "thank you very much for that question , chair . i 've got to say 'yes ' , in many regards . so , the key focus of the programme in the early stages was about improving access to specialist child and adolescent mental health services . we developed the windscreen model—or we lifted the windscreen model ."

# encode context the generation is conditioned on
input_ids = tokenizer.encode(sentence, return_tensors="tf")

# set seed to reproduce results. Feel free to change the seed though to get different results
tf.random.set_seed(0)

sample_outputs = gpt_model.generate(
    input_ids,
    do_sample=True,
    max_length=200,
    top_k=10,
    temperature=0.7,
    no_repeat_ngram_size=2,
    num_return_sequences=1,
)

text_output = [tokenizer.decode(x, skip_special_tokens=True) for x in sample_outputs]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


<IPython.core.display.Javascript object>

In [12]:
text_output

["thank you very much for that question, chair. i've got to say 'yes ', in many regards. so, the key focus of the programme in the early stages was about improving access to specialist child and adolescent mental health services. we developed the windscreen model—or we lifted the windscreen model. it was a huge undertaking, i.e., it would have required a lot of work. and we had to be very patient. there were many challenges in implementing the program, but the results were good. the first phase is the 'new' program. a new programme would be introduced in 2020-21. this would allow for the transition to the next phase of care, which is 'care for children at risk of suicide'. This programme was the culmination of a long project. 'Care for Children at Risk of Suicide' is a national, multi-generational initiative designed to provide mental and physical health care for a range of vulnerable people in Australia..\n\nThe"]

<IPython.core.display.Javascript object>

In [19]:
from nltk.tokenize import sent_tokenize

sent_tokenize(text_output[0])

['thank you very much for that question, chair.',
 "i've got to say 'yes ', in many regards.",
 'so, the key focus of the programme in the early stages was about improving access to specialist child and adolescent mental health services.',
 'we developed the windscreen model—or we lifted the windscreen model.',
 'it was a huge undertaking, i.e., it would have required a lot of work.',
 'and we had to be very patient.',
 'there were many challenges in implementing the program, but the results were good.',
 "the first phase is the 'new' program.",
 "a new programme would be introduced in 2020-21. this would allow for the transition to the next phase of care, which is 'care for children at risk of suicide'.",
 'This programme was the culmination of a long project.',
 "'Care for Children at Risk of Suicide' is a national, multi-generational initiative designed to provide mental and physical health care for a range of vulnerable people in Australia..",
 'The']

<IPython.core.display.Javascript object>

## Batch Upload

In [17]:
from db.dbv2 import Table, AugmentedTable, TrainTestTable

<IPython.core.display.Javascript object>

In [ ]:
dataset_type = "committee"

table = Table(dataset_type)
augmented_table = AugmentedTable(dataset_type)
train_test_table = TrainTestTable(dataset_type)

min_sent_tokens = 8
max_sent_tokens = 64

target_sentences_original = table.get_target_sentences()
target_sentences = [s[1] for s in target_sentences_original]
cleaned_target_sentences = []

for s in target_sentences:
    if len(word_tokenize(s)) > min_sent_tokens:
        shortened_sentence = " ".join(word_tokenize(s)[:max_sent_tokens])
        cleaned_target_sentences.append(shortened_sentence)

augmented_segments = Augmentor.augment_gpt2(
    cleaned_target_sentences[:2],
    fast=True,
    # multiply by 5 to account for 5 as a max segment
    max_seq_word_length=max_sent_tokens * 5,
    verbose=True,
)